In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pandas.stats.api import ols
%pylab inline
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

Populating the interactive namespace from numpy and matplotlib


In [9]:
listofcolumns = ['road_id', 'road_length', 'record_count', 'mean_SndMean', 'mean_lux', 'mean_acel', 'mean_isloud', 
                 'mean_isdark', 'mean_bumps', 'sum_isloud', 'sum_isdark', 'sum_bumps', 'rn_mean_SndMean', 'rn_mean_lux',
                 'rn_mean_acel', 'rn_mean_isloud', 'rn_mean_isdark', 'rn_mean_bumps', 'rn_sum_isloud', 
                 'rn_sum_isdark', 'rn_sum_bumps']


In [10]:
sensorXtrain = pd.DataFrame(np.load('x_train.npy'), columns= listofcolumns).convert_objects(convert_numeric=True)
sensorYtrain = pd.DataFrame(np.load('y_train.npy'), columns = ['Y'])

sensorXvalid = pd.DataFrame(np.load('x_valid.npy'), columns= listofcolumns).convert_objects(convert_numeric=True)
sensorYvalid = pd.DataFrame(np.load('y_valid.npy'), columns = ['Y'])

sensorXtest = pd.DataFrame(np.load('x_test.npy'), columns= listofcolumns).convert_objects(convert_numeric=True)
sensorYtest = pd.DataFrame(np.load('y_test.npy'), columns = ['Y'])

/Users/maxfacts/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':
/Users/maxfacts/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/maxfacts/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


# Logistic Classification

In [55]:
len(XmeanRNcatTrain)

816

In [127]:
XmeanRNcatTrain = sensorXtrain[['rn_mean_isloud','rn_mean_isdark','rn_mean_isdark',]]
XsumRNcatTrain = sensorXtrain[['rn_sum_isloud','rn_sum_isdark','rn_sum_isdark',]]
XmeanRNTrain = sensorXtrain[['rn_mean_acel', 'rn_mean_lux', 'rn_mean_SndMean']]
XmeanTrain = sensorXtrain[['mean_SndMean', 'mean_lux', 'mean_acel']]
Ytrain = np.load('y_train.npy')


LogTrain = [XmeanRNcatTrain, XsumRNcatTrain, XmeanRNTrain, XmeanTrain]
LogTrainScores = []
for  df in LogTrain:
    logclass = LogisticRegression()
    logclass.fit(np.asarray(df), Ytrain)
    LogTrainScores.append(logclass.score(np.asarray(df), Ytrain))


bestlogclass = LogisticRegression()
Xtrain =  np.asarray(LogTrain[LogTrainScores.index(max(LogTrainScores))])
bestlogclass.fit(X, Ytrain)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [147]:
LogTrainScores

[0.47549019607843135,
 0.53431372549019607,
 0.49877450980392157,
 0.54656862745098034]

In [146]:

Probs = pd.DataFrame(bestlogclass.predict_proba(X))
Probs['real'] = Ytrain
Probs['model'] = bestlogclass.predict(X)

# Probs['AccFlag'] = Probs

,0,1,2,3,real,model
0,0.395050,0.489048,0.039787,0.076115,2,1
1,0.856526,0.086281,0.036023,0.021170,0,0
2,0.377702,0.504301,0.045329,0.072668,1,1
3,0.329725,0.518786,0.074755,0.076734,2,1
4,0.307841,0.526146,0.086231,0.079782,2,1


# Logistic Test Set

In [129]:
XmeanRNcatTest = sensorXtest[['rn_mean_isloud','rn_mean_isdark','rn_mean_isdark',]]
XsumRNcatTest = sensorXtest[['rn_sum_isloud','rn_sum_isdark','rn_sum_isdark',]]
XmeanRNTest = sensorXtest[['rn_mean_acel', 'rn_mean_lux', 'rn_mean_SndMean']]
XmeanTest = sensorXtest[['mean_SndMean', 'mean_lux', 'mean_acel']]
Ytest = np.load('y_test.npy')
LogTest = [XmeanRNcatTest, XsumRNcatTest, XmeanRNTest, XmeanTest]

In [130]:
Xtest = LogTest[LogTrainScores.index(max(LogTrainScores))]

In [135]:
modelreturns = bestlogclass.predict(Xtest)

In [143]:
res = pd.DataFrame({'predicted': modelreturns, 'observed': Ytest})
res['rec'] = 1
res = res.pivot_table(index='observed', columns='predicted', values='rec', aggfunc=np.count_nonzero)
res1 = res / res.sum(1)

In [144]:
res

predicted,0,1
observed,,
0,38,131
1,3,165
2,3,34
3,NaN,35


In [148]:
res1.columns = ['Commerical', 'Multi-Family', 'Industrial', 'High Density Residential']

In [149]:
res1

,Commerical,Multi-Family,Industrial,High Density Residential
observed,,,,
0,0.224852,0.779762,NaN,NaN
1,0.017751,0.982143,NaN,NaN
2,0.017751,0.202381,NaN,NaN
3,NaN,0.208333,NaN,NaN
